In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
import os

import modules.logger_tool as logger
os.environ['LOG_NAME'] = 'Timetable'
os.environ['LOG_DIR'] = 'logs'
os.environ['LOG_LEVEL'] = 'INFO'

logging = logger.get_logger(os.environ['LOG_NAME'], log_level=os.environ['LOG_LEVEL'], log_path=os.environ['LOG_DIR'], log_file=os.environ['LOG_NAME'])


In [2]:
import modules.database.schemas.calendar_neo as calendar_neo
import modules.database.schemas.timetable_neo as timetable_neo
import modules.database.schemas.relationships.calendar_rels as cal_rels
import modules.database.schemas.relationships.timetable_rels as tt_rels
import modules.database.schemas.relationships.calendar_timetable_rels as cal_tt_rels

import modules.database.tools.xl_tools as xl
import modules.database.tools.neontology_tools as neon
import modules.database.tools.neo4j_driver_tools as driver_tools

import requests
from datetime import timedelta, datetime, date
import pandas as pd
from pydantic import ValidationError


In [3]:
db_name = os.environ['LOG_NAME']


In [4]:
url = f'http://{os.environ["BACKEND_URL"]}:{os.environ["BACKEND_PORT"]}/database/admin/stop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = f'http://{os.environ["BACKEND_URL"]}:{os.environ["BACKEND_PORT"]}/database/admin/drop-database'
data = {'db_name': db_name}
response = requests.post(url, json=data)
logging.info(response.text)

url = f'http://{os.environ["BACKEND_URL"]}:{os.environ["BACKEND_PORT"]}/database/admin/create-database'
params = {'db_name': db_name}
response = requests.post(url, params=params)
logging.info(response.text)

2024-07-07 19:31:45,579 INFO      : 40684991 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAsJA="]}
2024-07-07 19:31:47,637 INFO      : 40684991 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAsZA="]}
2024-07-07 19:31:49,727 INFO      : 40684991 > <module> >>> {"results":[{"columns":[],"data":[]}],"errors":[],"lastBookmarks":["FB:kcwQAAAAAAAAAAAAAAAAAAAAAckAspA="]}


In [5]:
# Initialize driver
driver = driver_tools.get_driver(database=db_name)

# Initialize connection
neon.init_neo4j_connection()


2024-07-07 19:31:49,888 INFO      : neo4j_driver_tools > get_driver >>> Connection successful
2024-07-07 19:31:50,006 INFO      : neontology_tools > init_neo4j_connection >>> Neontology connection initialized with host: 100.65.148.161, port: 7687, user: neo4j


In [6]:
def create_calendar(start_date, end_date, attach=False):
    # Dictionary to track created nodes to avoid duplicates
    created_years = {}
    created_months = {}
    created_weeks = {}
    created_days = {}

    # Variables to store the last node of each type
    last_year_node = None
    last_month_node = None
    last_week_node = None
    last_day_node = None

    # Dictionary to store the nodes and relationships
    calendar_nodes = []
    year_nodes = []
    month_nodes = []
    week_nodes = []
    day_nodes = []
    calendar_nodes = {
        'calendar_node': calendar_nodes,
        'calendar_year_nodes': year_nodes,
        'calendar_month_nodes': month_nodes,
        'calendar_week_nodes': week_nodes,
        'calendar_day_nodes': day_nodes
    }
    
    current_date = start_date

    if attach:
        calendar_node = calendar_neo.CalendarNode(
            calendar_id=f"calendar_{db_name}",
            calendar_name=db_name
        )
        neon.create_or_merge_neontology_node(calendar_node, database=db_name, operation='merge')
        calendar_nodes['calendar_node'] = calendar_node

    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        day = current_date.day
        iso_year, iso_week, iso_weekday = current_date.isocalendar()

        # Ensure year node is created for the Gregorian calendar year
        if year not in created_years:
            year_node = calendar_neo.CalendarYearNode(
                calendar_year_id=f"calendar_year_{year}",
                year=year,
                iso_year=str(year),
                localised_year_name=f"Year {year}"
            )
            neon.create_or_merge_neontology_node(year_node, database=db_name, operation='merge')
            calendar_nodes['year_nodes'].append(year_node)
            created_years[year] = year_node
            
            if attach:
                neon.create_or_merge_neontology_relationship(
                    cal_rels.CalendarIncludesYear(source=calendar_node, target=year_node),
                    database=db_name,
                    operation='merge'
                )

            if last_year_node:
                neon.create_or_merge_neontology_relationship(
                    cal_rels.YearFollowsYear(source=last_year_node, target=year_node),
                    database=db_name,
                    operation='merge'
                )
            last_year_node = year_node

        # Ensure month node is created
        month_key = f"{year}-{month}"
        if month_key not in created_months:
            month_node = calendar_neo.CalendarMonthNode(
                calendar_month_id=f"calendar_month_{year}_{month}",
                month=month,
                month_name=datetime(year, month, 1).strftime('%B'),
                iso_month=f"{year}-{month:02}",
                localised_month_name=f"{datetime(year, month, 1).strftime('%B')} {year}"
            )
            neon.create_or_merge_neontology_node(month_node, database=db_name, operation='merge')
            calendar_nodes['month_nodes'].append(month_node)
            created_months[month_key] = month_node

            # Check for the end of year transition for months
            if last_month_node:
                if month == 1 and last_month_node.month == 12 and int(last_month_node.calendar_month_id.split('_')[1]) == year - 1:
                    neon.create_or_merge_neontology_relationship(
                        cal_rels.MonthFollowsMonth(source=last_month_node, target=month_node),
                        database=db_name,
                        operation='merge'
                    )
                elif month == last_month_node.month + 1:
                    neon.create_or_merge_neontology_relationship(
                        cal_rels.MonthFollowsMonth(source=last_month_node, target=month_node),
                        database=db_name,
                        operation='merge'
                    )

            last_month_node = month_node

            neon.create_or_merge_neontology_relationship(
                cal_rels.YearIncludesMonth(source=year_node, target=month_node),
                database=db_name,
                operation='merge'
            )

        # Week node management
        week_key = f"{iso_year}-W{iso_week}"
        if week_key not in created_weeks:
            week_node = calendar_neo.CalendarWeekNode(
                calendar_week_id=f"calendar_week_{iso_year}_{iso_week}",
                week_number=iso_week,
                iso_week=f"{iso_year}-W{iso_week:02}",
                localised_week_name=f"Week {iso_week}, {iso_year}"
            )
            neon.create_or_merge_neontology_node(week_node, database=db_name, operation='merge')
            calendar_nodes['week_nodes'].append(week_node)
            created_weeks[week_key] = week_node

            if last_week_node and ((last_week_node.iso_week.split('-')[0] == str(iso_year) and last_week_node.week_number == iso_week - 1) or
                                (last_week_node.iso_week.split('-')[0] != str(iso_year) and last_week_node.week_number == 52 and iso_week == 1)):
                neon.create_or_merge_neontology_relationship(
                    cal_rels.WeekFollowsWeek(source=last_week_node, target=week_node),
                    database=db_name,
                    operation='merge'
                )
            last_week_node = week_node

            neon.create_or_merge_neontology_relationship(
                cal_rels.YearIncludesWeek(source=year_node, target=week_node),
                database=db_name,
                operation='merge'
            )

        # Day node management
        day_key = f"{year}-{month}-{day}"
        day_node = calendar_neo.CalendarDayNode(
            calendar_day_id=f"calendar_day_{year}_{month}_{day}",
            date=current_date,
            day_of_week=current_date.strftime('%A'),
            iso_day=f"{year}-{month:02}-{day:02}",
            localised_day_name=f"{current_date.strftime('%A')}, {current_date.strftime('%B')} {day}, {year}",
            is_weekend=current_date.weekday() > 4
        )
        neon.create_or_merge_neontology_node(day_node, database=db_name, operation='merge')
        calendar_nodes['day_nodes'].append(day_node)
        created_days[day_key] = day_node

        if last_day_node:
            neon.create_or_merge_neontology_relationship(
                cal_rels.DayFollowsDay(source=last_day_node, target=day_node),
                database=db_name,
                operation='merge'
            )
        last_day_node = day_node

        neon.create_or_merge_neontology_relationship(
            cal_rels.MonthIncludesDay(source=month_node, target=day_node),
            database=db_name,
            operation='merge'
        )

        neon.create_or_merge_neontology_relationship(
            cal_rels.WeekIncludesDay(source=week_node, target=day_node),
            database=db_name,
            operation='merge'
        )

        current_date += timedelta(days=1)
        
        return calendar_nodes

In [7]:
def create_calendar_and_timetable(data=None):
    # Check if data is None
    if data is None:
        raise ValueError("Data is required to create the calendar and timetable.")
    
    # Extract worksheets into dataframes
    school_df = data['schoollookup_df']
    terms_df = data['termslookup_df']
    weeks_df = data['weekslookup_df']
    days_df = data['dayslookup_df']
    periods_df = data['periodslookup_df']
    
    # Parse start and end dates for the timetable
    start_date = datetime.strptime(school_df[school_df['Identifier'] == 'AcademicYearStart']['Data'].iloc[0], '%Y-%m-%d')
    end_date = datetime.strptime(school_df[school_df['Identifier'] == 'AcademicYearEnd']['Data'].iloc[0], '%Y-%m-%d')
    school_id = school_df[school_df['Identifier'] == 'SchoolID']['Data'].iloc[0]

    # Create the calendar and retrieve nodes
    calendar_nodes = create_calendar(start_date, end_date, attach=True)

    # Create a dictionary to store the timetable nodes
    timetable_nodes = {
        'timetable_node': None,
        'academic_year_nodes': [],
        'academic_term_nodes': [],
        'academic_week_nodes': [],
        'academic_day_nodes': [],
        'academic_period_nodes': []
    }
    
    # Create AcademicTimetable Node
    academic_timetable_node = timetable_neo.AcademicTimetableNode(
        academic_timetable_id=f"{school_id}_{start_date.year}_{end_date.year}",
        academic_timetable_name=f"{start_date.year} to {end_date.year}",
        academic_timetable_start_date=start_date,
        academic_timetable_end_date=end_date
    )
    neon.create_or_merge_neontology_node(academic_timetable_node, database=db_name, operation='merge')
    timetable_nodes['timetable_node'] = academic_timetable_node

    # Create AcademicYear nodes for each year within the range
    for year in range(start_date.year, end_date.year + 1):
        academic_year_node = timetable_neo.AcademicYearNode(
            academic_year_id=f"AcademicYear_{school_id}_{year}",
            academic_year=str(year)
        )
        neon.create_or_merge_neontology_node(academic_year_node, database=db_name, operation='merge')
        timetable_nodes['academic_year_nodes'].append(academic_year_node)
        neon.create_or_merge_neontology_relationship(
            tt_rels.AcademicTimetableHasAcademicYear(source=academic_timetable_node, target=academic_year_node),
            database=db_name, operation='merge'
        )

        # Link the academic year with the corresponding calendar year node
        for year_node in calendar_nodes['year_nodes']:
            if year_node.year == year:
                neon.create_or_merge_neontology_relationship(
                    cal_tt_rels.CalendarYearIsAcademicYear(source=year_node, target=academic_year_node),
                    database=db_name, operation='merge'
                )
                neon.create_or_merge_neontology_relationship(
                    cal_tt_rels.AcademicYearIsCalendarYear(source=academic_year_node, target=year_node),
                    database=db_name, operation='merge'
                )
                break

    # Create Term and TermBreak nodes linked to AcademicYear
    for _, term_row in terms_df.iterrows():
        term_node_class = timetable_neo.AcademicTermNode if term_row['TermType'] == 'Term' else timetable_neo.AcademicTermBreakNode
        term_node = term_node_class(
            term_id=f"{term_row['TermType']}_{term_row['TermName'].replace(' ', '')}",
            term_name=term_row['TermName'],
            term_number=term_row['TermName'].split()[-1],
            term_start_date=datetime.strptime(term_row['Date'], '%Y-%m-%d') if term_row['DateType'] == 'Start' else None,
            term_end_date=datetime.strptime(term_row['Date'], '%Y-%m-%d') if term_row['DateType'] == 'End' else None
        )
        neon.create_or_merge_neontology_node(term_node, database=db_name, operation='merge')
        timetable_nodes['academic_term_nodes'].append(term_node)

        # Link term node to the correct academic year
        term_years = set([term_node.term_start_date.year, term_node.term_end_date.year])
        for academic_year_node in calendar_nodes['year_nodes']:
            if academic_year_node.year in term_years:
                relationship_class = tt_rels.AcademicYearHasAcademicTerm if term_row['TermType'] == 'Term' else tt_rels.AcademicYearHasAcademicTermBreak
                neon.create_or_merge_neontology_relationship(
                    relationship_class(source=academic_year_node, target=term_node),
                    database=db_name, operation='merge'
                )

    # Create Week nodes
    for _, week_row in weeks_df.iterrows():
        week_node_class = timetable_neo.HolidayWeekNode if week_row['WeekType'] == 'Holiday' else timetable_neo.AcademicWeekNode
        week_node = week_node_class(
            week_id=f"{week_row['WeekType']}_{week_row['WeekNumber']}",
            week_number=week_row['WeekNumber'],
            week_start_date=datetime.strptime(week_row['WeekStart'], '%Y-%m-%d'),
            week_type=week_row['WeekType']
        )
        neon.create_or_merge_neontology_node(week_node, database=db_name, operation='merge')
        timetable_nodes['academic_week_nodes'].append(week_node)

        # Link week node to the correct academic term
        for term_node in terms_df.itertuples():
            if term_node.term_start_date <= week_node.week_start_date <= term_node.term_end_date:
                relationship_class = tt_rels.AcademicTermHasAcademicWeek if week_row['WeekType'] != 'Holiday' else tt_rels.AcademicTermHasHolidayWeek
                neon.create_or_merge_neontology_relationship(
                    relationship_class(source=term_node, target=week_node),
                    database=db_name, operation='merge'
                )
                break

        # Link week node to the correct academic year
        for academic_year_node in calendar_nodes['year_nodes']:
            if academic_year_node.year == week_node.week_start_date.year:
                relationship_class = tt_rels.AcademicYearHasAcademicWeek if week_row['WeekType'] != 'Holiday' else tt_rels.AcademicYearHasHolidayWeek
                neon.create_or_merge_neontology_relationship(
                    relationship_class(source=academic_year_node, target=week_node),
                    database=db_name, operation='merge'
                )
                break

    # Create Day nodes
    for _, day_row in days_df.iterrows():
        day_node_class = {
            'Academic': timetable_neo.AcademicDayNode,
            'Holiday': timetable_neo.HolidayDayNode,
            'OffTimetable': timetable_neo.OffTimetableDayNode,
            'Staff': timetable_neo.StaffDayNode
        }[day_row['DayType']]
        day_node = day_node_class(
            day_id=f"{day_row['DayType']}_{day_row['Date']}",
            day_number=day_row['Day'],
            date=datetime.strptime(day_row['Date'], '%Y-%m-%d'),
            day_of_week=day_row['DayOfWeek'],
            day_type=day_row['DayType']
        )
        neon.create_or_merge_neontology_node(day_node, database=db_name, operation='merge')
        timetable_nodes['academic_day_nodes'].append(day_node)

        # Link day node to the correct academic week
        for week_node in calendar_nodes['week_nodes']:
            if week_node.week_start_date <= day_node.date <= (week_node.week_start_date + timedelta(days=6)):
                relationship_class = tt_rels.AcademicWeekHasAcademicDay if day_row['DayType'] == 'Academic' else tt_rels.AcademicWeekHasHolidayDay
                neon.create_or_merge_neontology_relationship(
                    relationship_class(source=week_node, target=day_node),
                    database=db_name, operation='merge'
                )
                break

        # Link day node to the correct academic term
        for term_node in terms_df.itertuples():
            if term_node.term_start_date <= day_node.date <= term_node.term_end_date:
                relationship_class = tt_rels.AcademicTermHasAcademicDay if day_row['DayType'] == 'Academic' else tt_rels.AcademicTermHasHolidayDay
                neon.create_or_merge_neontology_relationship(
                    relationship_class(source=term_node, target=day_node),
                    database=db_name, operation='merge'
                )
                break

        # Create Period nodes for each day
        for _, period_row in periods_df.iterrows():
            period_node_class = {
                'Lesson': timetable_neo.AcademicPeriodNode,
                'Registration': timetable_neo.RegistrationPeriodNode,
                'Break': timetable_neo.BreakPeriodNode
            }[period_row['PeriodType']]
            period_node = period_node_class(
                period_id=f"{period_row['PeriodType']}_{day_node.day_id}_{period_row['Time'].replace(':', '')}",
                period_name=period_row['PeriodName'],
                period_start_time=datetime.strptime(period_row['Time'], '%H:%M:%S') if period_row['TimeType'] == 'Start' else None,
                period_end_time=datetime.strptime(period_row['Time'], '%H:%M:%S') if period_row['TimeType'] == 'End' else None,
                period_day_of_week=day_node.day_of_week,
                period_day_type=day_node.day_type
            )
            neon.create_or_merge_neontology_node(period_node, database=db_name, operation='merge')
            timetable_nodes['academic_period_nodes'].append(period_node)
            
            relationship_class = {
                'Lesson': tt_rels.AcademicDayHasAcademicPeriod,
                'Registration': tt_rels.AcademicDayHasRegistrationPeriod,
                'Break': tt_rels.AcademicDayHasBreakPeriod
            }[period_row['PeriodType']]
            
            neon.create_or_merge_neontology_relationship(
                relationship_class(source=day_node, target=period_node),
                database=db_name, operation='merge'
            )

    return {
        'calendar_nodes': calendar_nodes,
        'timetable_nodes': timetable_nodes
    }

In [8]:
# Get the Excel file
nodes = create_calendar_and_timetable(data=xl.create_dataframes(os.getenv("EXCEL_TIMETABLE_FILE")))


TypeError: strptime() argument 1 must be str, not datetime.datetime